Link al repositorio de Github: https://github.com/100461170/AA_practica_2_grupo_10

Para subir los archivos y demás a lo largo de los diferentes cuadernos hemos estado usando Drive ya que es muy rápido y cómodo, en vez de subir los archivos manualmente

Autores: Alejandro Díaz Cuéllar 100472173 y Tomás Mendizábal 100461170

# **Obtener los datos**

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np

# Si quieres subir el archivo wind_ava.csv desde tu ordenador
#uploaded = files.upload()

# Montar el drive para incluir el archivo desde ahí
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
seed = 100472173
# Leer el archivo CSV utilizando Pandas
df = pd.read_csv("/content/drive/MyDrive/AA_P2_Grupo10/stars_data.csv")

# **1.- Programar nuestro propio KMeans**

## 1.1.- Implementación

In [ ]:
# Funcion KMeans. Al tener que ser una funcion, directamente hacemos que sea el fit
# del KMeans de ScikitLearn
def my_kmeans(data, n_clusters, good_init=False):
  # Si el good_init está activado, haremos 10 iteraciones
  if good_init:
    iterations = 10
  else:
    iterations = 1
  # Para almacenar los resultados e inertias de cada iteracion
  inertias = []
  results = []
  for i in range(iterations):
    converger = False
    # Cogemos los centroides iniciales seleccionando aleatoriamente n_clusters
    # instancias de los datos
    centroids = data[np.random.choice(data.shape[0], n_clusters, replace=False)]
    while not converger:
      # Calculamos la distancia euclidea de cada elemento del dataset a cada
      # centroide
      distances = np.linalg.norm(data[:, np.newaxis] - centroids, axis=2)
      # La label de cada centroide será el centroide del que estén mas cerca
      labels = np.argmin(distances, axis=1)
      # Calculamos los nuevos centroides cogiendo la media de los elementos
      # asignados a cada label
      new_centroids = np.array([data[labels == k].mean(axis=0) for k in range(n_clusters)])
      # Miramos si ha convergido
      if np.allclose(centroids, new_centroids):
          converger = True
      centroids = new_centroids
    results.append(labels)
    # Calculamos la inercia
    inertia = 0
    for k in range(len(centroids)):
        inertia += np.sum((data[labels == k] - centroids[k])**2)
    inertias.append(inertia)
  # Devolvemos los centroides cuyo modelo tenga menos inercia
  winner = inertias.index(min(inertias))
  return results[winner]

## 1.2.- Pruebas

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score, completeness_score

X, y = make_blobs(n_samples=300, centers=4, cluster_std=0.60)

# Ajustar tu implementación de KMeans
labels_custom = my_kmeans(X, 4)

# Ajustar el KMeans de scikit-learn
sklearn_kmeans = KMeans(n_clusters=4)
labels_sklearn = sklearn_kmeans.fit_predict(X)

# Calcular algunas métricas de evaluación
silhouette_custom = silhouette_score(X, labels_custom)
silhouette_sklearn = silhouette_score(X, labels_sklearn)

ari_custom = adjusted_rand_score(y, labels_custom)
ari_sklearn = adjusted_rand_score(y, labels_sklearn)

completeness_custom = completeness_score(y, labels_custom)
completeness_sklearn = completeness_score(y, labels_sklearn)

# Imprimir las métricas
print("Métricas de evaluación para tu implementación de KMeans:")
print("Silhouette Score:", silhouette_custom)
print("Adjusted Rand Index:", ari_custom)
print("Completeness Score:", completeness_custom)
print()

print("Métricas de evaluación para KMeans de scikit-learn:")
print("Silhouette Score:", silhouette_sklearn)
print("Adjusted Rand Index:", ari_sklearn)
print("Completeness Score:", completeness_sklearn)

Métricas de evaluación para tu implementación de KMeans:
Silhouette Score: 0.7619686555881909
Adjusted Rand Index: 1.0
Completeness Score: 1.0

Métricas de evaluación para KMeans de scikit-learn:
Silhouette Score: 0.7619686555881909
Adjusted Rand Index: 1.0
Completeness Score: 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


```
// FALTA HACER EL ANÁLISIS DE RESULTADOS AQUÍ
```

# **2.- Uso de 2 algoritmos de Clustering**

## 2.1.- Transformación de los datos

Primero vamos a transformar los colores del dataset para que sean más uniformes

In [ ]:
# Establecemos una unificación de colores:
mapping = {   # Diccionario con formato Target : List of current values
    'Red': ['Red'],
    'Orange-Red': ['Orange-Red'],
    'Orange': ['Orange'],
    'Yellow-Orange': ['Pale yellow orange'],
    'Yellow': ['yellowish', 'Yellowish'],
    'White-Yellow': ['Yellowish White', 'yellow-white', 'White-Yellow'],
    'White': ['White', 'Whitish', 'white'],
    'Blue-White': ['Blue White', 'Blue white', 'Blue-white', 'Blue-White'],
    'Blue': ['Blue'],
}
# Mapeamos los datos
for target_color, source_colors in mapping.items():
  df.loc[df.Color.isin(source_colors), 'Color'] = target_color

Codificamos el dataframe de dos formas, con One Hot Encoding y con Ordinal Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# ONE HOT ENCODING
# Seleccionar las columnas con variables categóricas
categorical_cols = ['Spectral_Class', 'Color']
encoder = OneHotEncoder(sparse=False, drop='first')
# Ajustar y transformar las variables categóricas
encoded_cols = encoder.fit_transform(df[categorical_cols])
# Convertir la salida a un DataFrame
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))
# Concatenar los DataFrames codificados con el DataFrame original
df_OH = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

# ORDINAL ENCODING
df_OR = df.copy()
colors = list(mapping.keys())
spectral = ['O', 'B', 'A', 'F', 'G', 'K', 'M']
# Crear un codificador OrdinalEncoder con el orden deseado
encoder = OrdinalEncoder(categories=[spectral, colors])
# Ajustar y transformar los datos
encoded_data = encoder.fit_transform(df_OR[categorical_cols])
# Reemplazar las columnas categóricas originales con las codificadas
df_OR['Spectral_Class'] = encoded_data[:, 0]
df_OR['Color'] = encoded_data[:, 1]

print(df_OH)
print(df_OR)

     Temperature              L          R    A_M  Spectral_Class_B  \
0           3068       0.002400     0.1700  16.12               0.0   
1           3042       0.000500     0.1542  16.60               0.0   
2           2600       0.000300     0.1020  18.70               0.0   
3           2800       0.000200     0.1600  16.65               0.0   
4           1939       0.000138     0.1030  20.06               0.0   
..           ...            ...        ...    ...               ...   
235        38940  374830.000000  1356.0000  -9.93               0.0   
236        30839  834042.000000  1194.0000 -10.63               0.0   
237         8829  537493.000000  1423.0000 -10.73               0.0   
238         9235  404940.000000  1112.0000 -11.23               0.0   
239        37882  294903.000000  1783.0000  -7.80               0.0   

     Spectral_Class_F  Spectral_Class_G  Spectral_Class_K  Spectral_Class_M  \
0                 0.0               0.0               0.0           

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Ahora extraemos dos componentes PCA

```
// NO SÉ SI LO DE APLICAR STANDARD ESTÁ BIEN POR EL TEMA DE LAS VARIABLES CODIFICADAS
```

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_OH_scaled = scaler.fit_transform(df_OH)
X_OR_scaled = scaler.fit_transform(df_OR)

# Creamos un objeto PCA con 2 componentes
pca = PCA(n_components=2)

# Aplicamos PCA a los datos estandarizados
X_OH_PCA = pca.fit_transform(X_OH_scaled)
X_OR_PCA = pca.fit_transform(X_OR_scaled)

## 2.2.- Algoritmo 1: KMeans

Como en el enunciado se menciona que normalmente las estrellas se pueden clasificar en 6 clases, y dado que necesitamos un número arbitrario de clusters para KMeans, vamos a aprovechar y a usar n_clusters = 6.

In [ ]:
kmeans = KMeans(n_clusters=6)

# ONE HOT ENCODING
labels_OH = kmeans.fit_predict(X_OH_PCA)

# ORDINAL ENCODING
labels_OR = kmeans.fit_predict(X_OR_PCA)

# Calcular la inercia y la puntuación de silueta para las etiquetas de one-hot encoding
inertia_OH = kmeans.inertia_
silhouette_score_OH = silhouette_score(X_OH_PCA, labels_OH)

# Calcular la inercia y la puntuación de silueta para las etiquetas de ordinal encoding
inertia_OR = kmeans.inertia_
silhouette_score_OR = silhouette_score(X_OR_PCA, labels_OR)

print("Inertia (One-Hot Encoding):", inertia_OH)
print("Silhouette Score (One-Hot Encoding):", silhouette_score_OH)
print("Inertia (Ordinal Encoding):", inertia_OR)
print("Silhouette Score (Ordinal Encoding):", silhouette_score_OR)

Inertia (One-Hot Encoding): 80.70203003813528
Silhouette Score (One-Hot Encoding): 0.6972556055210379
Inertia (Ordinal Encoding): 80.70203003813528
Silhouette Score (Ordinal Encoding): 0.6605298437727174


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(




```
// HAY QUE INTERPRETAR LOS DATOS Y AHORA MISMO NO SÉ
```



## 2.3.- Algoritmo 2: DBSCAN